In [9]:
import numpy as np
import pandas as pd
from numpy.random import rand
import gzip
import json
import ast
from scipy.cluster.vq import kmeans2
import matplotlib.pyplot as plt
import random
from scipy.sparse import coo_matrix
from scipy.sparse import lil_matrix
from numpy.random import rand
from scipy.sparse import csr_matrix, isspmatrix_csr
from pandas.api.types import CategoricalDtype

In [21]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')
df = getDF('reviews_Musical_Instruments.json.gz')
#df  is the whole table of data, we need to act on this
# very imp code : https://stackoverflow.com/questions/49405101/grouping-all-rows-of-a-pandas-dataframewith-many-columns-with-the-same-value-i
df = df[['reviewerID','asin']]
#df1 = df.astype(str).groupby('reviewerID').agg(','.join).reset_index()
#df1 will print the whole 2 col list of items and reviewers. Now assign 1 as value
df_copy = df.copy()
df_copy["value"] = 1
df_copy# this prints the best col format. Now to convert to matrix:

reviewer = CategoricalDtype(sorted(df_copy["reviewerID"].unique()), ordered=True)
product = CategoricalDtype(sorted(df_copy["asin"].unique()), ordered=True)

row = df_copy["reviewerID"].astype(reviewer).cat.codes
col = df_copy["asin"].astype(product).cat.codes

sparse_matrix = csr_matrix((df_copy["value"], (row, col)),
                           shape=(reviewer.categories.size, product.categories.size))
sparse_matrix[:1000,:1000].toarray()
#A.tocsr()[:,select_ind].todense()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
def createRandomSortedList(num, start, end): 
    arr = [] 
    tmp = random.randint(start, end) 
      
    for x in range(num): 
          
        while tmp in arr: 
            tmp = random.randint(start, end) 
              
        arr.append(tmp) 
    return arr 

myrow = createRandomSortedList(10, 1, 1000)
mycol = createRandomSortedList(10, 1, 1000)

row  = np.array(myrow)
col  = np.array(mycol)
newdata = np.array(np.ones(10))
data = np.array(newdata)
A = coo_matrix((data, (row, col)), shape=(1000, 1000)).tocsr()# this matrix A is the input to the svd

# Now the svd begins, look at the slide pg 2, in the folder for the algorithm

k = 300
l = k+20
m = 1000
n = 1000
# now in the first step we choose k = 100, l = 102, i = 2, A is considered mxn , so m = 10000, n= 5000
# In the first step, Using a random number generator, form a real n × l matrix G whose entries are independent and identically distributed
# Gaussian random variables of zero mean and unit variance
mu = 0
sigma = 1.0
G = np.random.normal(mu, sigma, (n,l))
#Compute B = AG (B ∈ R^ m×l )
#B = np.matmul(A,G)#---------------------------------------------------------first multiplication by A
B = []
for i in range(l):
    v = G[:,i]
    x = A.dot(v)
    B.append(x)
B = np.array(B).T
A = None # no more needed
G = None# not reqd anymore
X, lamda, Yt = np.linalg.svd(B, full_matrices=True)

lamda = None
Yt =  None
Q = X[:, : k] #(Q ∈ R m×k )
#Qt = Q.transpose()

#T = np.matmul(Qt,A)#-------------------------------2nd mult by A, only these two are there in this algo
#Newsparse_matrix = csr_matrix((df_copy["value"], (col, row)),shape=(reviewer.categories.size, product.categories.size))
At = coo_matrix((data, (col, row)), shape=(1000, 1000)).tocsr()
T = []
for i in range(k):
    v = Q[:,i]
    x = At.dot(v)
    T.append(x)
T = np.array(T)
At = None# not needed anymore
row = None
col = None
W, singlr, Vt = np.linalg.svd(T, full_matrices=True)
T = None# not needed anymore
Vt = None# not needed for the smaller dim matrix creation
U = np.matmul(Q,W)
Q = None
W = None
final = np.multiply(U,singlr)
final

array([[-6.28535630e-17, -2.59979780e-17, -4.98728140e-17, ...,
         0.00000000e+00, -5.31556579e-18,  9.49931888e-18],
       [ 8.22847682e-17, -6.86097326e-17,  3.56166047e-17, ...,
         0.00000000e+00,  2.33380897e-19, -9.66930708e-19],
       [-3.33941937e-17, -1.62988901e-16, -1.44993445e-16, ...,
         0.00000000e+00,  1.10404491e-18,  1.08766129e-18],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [26]:
# we test with bin matrices which has 300 columns corresponding to dimension of points
# we take 10 rows which is the number of accounts, this value should be around 10^6 to be set later
#mu = 0
#sigma = 1.0
#A = np.random.normal(mu, sigma, (1000,300))
A = final
#Now do the 2-means and get the output.
#centroid, label = kmeans2(A, 2, minit='points')
#centroid will be two arrays each of 10 dimension for the two clusters
# label[i] is the code or index of the centroid the ith observation is closest to.

# Now we write the function that partitons A into A0 and A1
# A0 and A1 are lists which will contain only the indices of the rows which are supposed to be in A0 and A1
# partition takes an arr which means it contains the matrix made up of A[arr[i]]
# we need to form matrix from the rows of A as mentioned in arr
arr = list(range(1000))# later this should be list(range(10**6))
def partition(arr):
    matrix = []
    for i in range(len(arr)):
        matrix.append(A[i,:])
    matrix = np.array(matrix)
    centroid, label = kmeans2(matrix, 2, minit='points')
    A0,A1 = [],[]
    for i in range(len(arr)):# this is the num of the rows, i.e. the num of accts
        if label[i] == 0:
            A0.append(i)# means A[i] should be in this as a row
        else:
            A1.append(i)# means A[i] should be in this as a row
    #A0 = np.array(A0)
    #A1 = np.array(A1)
    return (A0,A1)# returns two lists so that x = A0[i], A[x] will give the reqd row

tau = 3
# data contains lists which contain indices i. A[i] will be the reqd rows, to start with data = [0,...,9]
class Node:
    def __init__ (self, data, parent = None):
        #self.width = width
        #self.height = height
        self.parent = parent
        self.children = []
        self.data = data

    @property
    def level (self):
        return 0 if not self.parent else 1 + self.parent.level

    def split (self):
        #if self.children == []: return
        numrow = len(self.data)
        if numrow <= tau:
            self.children = []
            return
        A1,A2 = partition(self.data)
        self.children = [Node(A1, self),Node(A2, self)]
        
    def splitUntilLevel (self, maxLevel):
        if maxLevel <= self.level: return
        self.split ()
        for child in self.children: child.splitUntilLevel (maxLevel)

    def sizeofNodes(self):
        s = [len(self.data)]
        for child in self.children:
            s.append(child.sizeofNodes())
        #print(s)
        return s
    
N = Node(list(range(1000)))
N.splitUntilLevel (5)
print(N.sizeofNodes())

[1000, [10, [4, [1], [3]], [6, [3], [3]]], [990, [10, [6, [4, [1], [3]], [2]], [4, [3], [1]]], [980, [10, [7, [3], [4]], [3]], [970, [10, [5], [5]], [960, [10], [950]]]]]]
